### DATA ANALYSIS PROJECT 


In [3]:
import pyodbc
from dotenv import dotenv_values 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pyodbc'

In [ ]:
environment_variables = dotenv_values('.env')

database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [ ]:
connection = pyodbc.connect(connection_string)